In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnull
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import when


# Utilisation de la base de données Hive, et charge les données de la table

In [2]:
spark = SparkSession.builder \
    .appName("Categorie") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()


In [3]:
spark.sql("USE concessionnaire")
client_immat_df = spark.sql("SELECT * FROM client_immatriculation_categorie")

# Afficher un aperçu des données

In [4]:
client_immat_df.show(20, truncate=False)

+---------------+---+-----+----+-------------------+---------------+---------------+----------+---------+-----------+--------+--------+-------+--------+--------+-------+---------+
|immatriculation|age|sexe |taux|situationfamilliale|nbenfantacharge|deuxiemevoiture|marque    |puissance|longueur   |nbplaces|nbportes|couleur|occasion|prix    |modele |categorie|
+---------------+---+-----+----+-------------------+---------------+---------------+----------+---------+-----------+--------+--------+-------+--------+--------+-------+---------+
|0 NQ 98        |50 |Femme|243 |En Couple          |0              |true           |Volkswagen|55       |courte     |5       |3       |blanc  |true    |8540.0  |Polo   |citadine |
|0 PW 98        |58 |Homme|502 |Célibataire        |0              |false          |Peugeot   |75       |courte     |5       |5       |noir   |false   |13750.0 |1007   |citadine |
|1 AK 80        |35 |Homme|1127|Célibataire        |0              |false          |Audi      |75   

In [5]:
client_immat_df.describe().show()

+-------+---------------+------------------+-----+-----------------+-------------------+------------------+---------------+------+------------------+-----------+--------+-------------------+-------+------------------+-----------------+---------+
|summary|immatriculation|               age| sexe|             taux|situationfamilliale|   nbenfantacharge|deuxiemevoiture|marque|         puissance|   longueur|nbplaces|           nbportes|couleur|              prix|           modele|categorie|
+-------+---------------+------------------+-----+-----------------+-------------------+------------------+---------------+------+------------------+-----------+--------+-------------------+-------+------------------+-----------------+---------+
|  count|          98816|             98816|98816|            98816|              98816|             98816|          98816| 98816|             98816|      98816|   98816|              98816|  98816|             98816|            98816|    98816|
|   mean|       

In [6]:

# Vérification des valeurs manquantes
missing_counts = client_immat_df.select([isnull(col(c)).alias(c) for c in client_immat_df.columns]).groupby().sum().show()


++
||
++
||
++



# Préparation des données

In [7]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

# Sélection des colonnes nécessaires
df_reduit = client_immat_df.select("age", "sexe", "taux", "situationfamilliale", "nbenfantacharge", "deuxiemevoiture", "categorie")

# Encodage des colonnes catégoriques
indexers = [
    StringIndexer(inputCol="sexe", outputCol="sexe_index"),
    StringIndexer(inputCol="situationfamilliale", outputCol="situation_index"),
    StringIndexer(inputCol="deuxiemevoiture", outputCol="deuxiemevoiture_num"),
    StringIndexer(inputCol="categorie", outputCol="categorie_index")
    # nbenfantacharge
]

# Nomaliser ou rendre categorielle age et taux

In [8]:

pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df_reduit).transform(df_reduit)

# Création de la colonne `features`
assembler = VectorAssembler(
    inputCols=["age", "sexe_index", "taux", "situation_index", "nbenfantacharge", "deuxiemevoiture_num"],
    outputCol="features"
)

df_final = assembler.transform(df_indexed)
df_final.printSchema()


root
 |-- age: integer (nullable = true)
 |-- sexe: string (nullable = true)
 |-- taux: integer (nullable = true)
 |-- situationfamilliale: string (nullable = true)
 |-- nbenfantacharge: integer (nullable = true)
 |-- deuxiemevoiture: string (nullable = true)
 |-- categorie: string (nullable = true)
 |-- sexe_index: double (nullable = false)
 |-- situation_index: double (nullable = false)
 |-- deuxiemevoiture_num: double (nullable = false)
 |-- categorie_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [9]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Division des données
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

In [10]:

# Création du modèle Random Forest
rf = RandomForestClassifier(
    labelCol="categorie_index",  # La cible encodée
    featuresCol="features",  # Les vecteurs
    numTrees=20,  # Nombre d'arbres
    maxDepth=5,  # Profondeur maximale
    seed=42
)


In [11]:
# Entraînement
rf_model = rf.fit(train_data)

In [12]:
# Prédiction sur l'ensemble de test
predictions = rf_model.transform(test_data)

In [ ]:
# Évaluation
evaluator = MulticlassClassificationEvaluator(
    labelCol="categorie_index", 
    predictionCol="prediction", 
    metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.7228330350294042
